# Spectrum detection algorithm

### Obtain power spectrum from SoapySDR devices (AIR-T, RTL-SDR, Airspy, SDRplay, HackRF, bladeRF, USRP, LimeSDR, etc.)

### link [soapy_power](https://github.com/xmikos/soapy_power)

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
#
#https://github.com/xmikos/soapy_power
# version 1.1
# Title: Spectrum sensing
# Author: David Góez
# UdeA-IDLab research group
# 2023/7


from datetime import datetime
from datetime import time

import SoapySDR
import simplesoapy
import numpy
import subprocess
import csv
from IPython.display import clear_output
import io
from datetime import datetime


In [29]:
def execute_soapy_power(cont):
    command = "soapy_power --force-rate -r 50M --antenna=RX --gain=30 -f 3400M:3800M -b 2048 -n 1000  -u 1 --remove-dc"
    try:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        output, _ = process.communicate()
        output_str = output.decode("utf-8")
        fecha = output_str[0:20]

        with open("/home/david-imec/gnuradio_test/udea_sensing/output_"+ fecha +".csv", "w", newline="") as csvfile:
            writer = csv.writer(csvfile)
            lines = output_str.strip().split(",")
            writer.writerow(lines)
        # clear_output()
        
    except subprocess.CalledProcessError as e:
        print("Error executing SoapyPower:", e)
        

In [30]:
def main(fh1,fh2):
    flag = True
    sensing=True
    cont = 0
    while flag:
        fecha_hora1 = fh1   # 
        fecha_hora2 = fh2   # 

        if datetime.now() > fecha_hora1 and datetime.now() < fecha_hora2:
            hora1 = time(10, 0,0,0)  # First time (10:00)
            hora2 = time(2, 0,0,0)   # Second time (02:00)

            if hora1.hour > hora2.hour:
        # Adjust the hour of the first time by adding 24
                adjusted_hour1_ = (hora1.hour + 24) % 24
                adjusted_hora1 = time(adjusted_hour1_, hora1.minute, hora1.second)
            else:
                adjusted_hora1 =  hora1
        
            fecha_hora_actual = datetime.now()  # get time and date data
            hora_actual = fecha_hora_actual.time()
            if hora_actual > hora1 and hora_actual > hora2:
                execute_soapy_power(cont)
                if sensing==True:
                    print("Sensing", end='')
                    sensing=False
                else:
                    print(".", end="", flush=True)
                cont+=1
            else:
                print("wait time")
                sensing=True
        elif datetime.now() > fecha_hora2:
            flag = False
    return cont

# Sensing time control

In [31]:
if __name__ == "__main__":
    # lists the devices found
    devices = SoapySDR.Device.enumerate()

    if devices:
        print(simplesoapy.detect_devices(as_string=True))
#------------------------------------------------------------------------
        fecha_hora1_ = datetime(2023, 7, 24, 14, 38)   # 
        fecha_hora2_ = datetime(2023, 7, 24, 14, 39)   #
#------------------------------------------------------------------------
        nd = main(fecha_hora1_ ,fecha_hora2_ )
        print(".......spectrum data ok --> total: ",nd)
    else:
        print(".......Error, radio not detected......")

['driver=uhd, serial=310A880, label=B200 310A880']
Sensing..........................spectrum data ok --> total:  20
